# Demonstration of Creating and Running a NATS input file from IFF data
## Read in IFF Data using PARA-ATM

In [1]:
from paraatm.io import iff, gnats, utils
from paraatm.io.iff import read_iff_file


#iff_fname = 'IFF_SFO_ASDEX_ABC123.csv'
iff_fname = 'IFF_SFO+ASDEX_20190511_080104_86221.csv'

iff_data =  read_iff_file(iff_fname,record_types=[2,3,4,8])
print(iff_data[3].columns)
print(len(iff_data[3]))

Index(['recType', 'time', 'fltKey', 'bcnCode', 'cid', 'Source', 'msgType',
       'callsign', 'recTypeCat', 'latitude', 'longitude', 'altitude',
       'significance', 'coord1Accur', 'coord2Accur', 'altAccur', 'tas',
       'heading', 'rocd', 'altQualifier', 'altIndicator', 'trackPtStatus',
       'leaderDir', 'scratchPad', 'msawInhibitInd', 'assignedAltString',
       'controllingFac', 'controllingSeg', 'receivingFac', 'receivingSec',
       'activeContr', 'primaryContr', 'kybrdSubset', 'kybrdSymbol', 'adsCode',
       'opsType', 'airportCode', 'trackNumber', 'tptReturnType', 'modeSCode'],
      dtype='object')
1030410


## Clean callsign list of UNKN, nan, 1200, and ground operations vehicles

In [2]:
#Note not necessary for sample file and will produce an error
callsigns = iff_data[3].callsign.unique()
cs = callsigns.tolist()
cs.remove('UNKN') # remove 'UNKN'
cs = [x for x in cs if (str(x) != 'nan')] #remove nan
cs = [x for x in cs if not x.startswith('OPS')] #remove ground operation vehicles
cs.remove('1200')

## Start NATS Simulation in the background

In [3]:
from gnats_gate_to_gate import GateToGate
natsSim = GateToGate()

## Create FlightPlanSelector object

In [4]:
from FlightPlanSelector import FlightPlanSelector
dirPath = natsSim.DIR_share
fpath = dirPath + '/tg/trx/TRX_07132005_noduplicates_crypted'
f=FlightPlanSelector(natsSim,fname=fpath)

## Generate NATS TRX flightplans

In [7]:
import os
from iff_functions import (get_departure_airport_from_iff,
                           get_arrival_airport_from_iff,
                           get_departure_gate_and_rwy_from_iff,
                           get_arrival_gate_and_rwy_from_iff,
                           random_airport_gate_and_rwy,
                           check_if_flight_has_departed,
                           check_if_flight_landing_in_dataset)

iffBaseAirport= "KSFO"

#Filenames to write modified trx and mfl files to. Need full path because GNATS wrapper changes the directory to the GNATS directory.
home_env = os.environ.get('HOME')
trx_dir = '/home/edecarlo/para-atm-collection/miscellaneous/gnats-fpgen'
results_dir = '/home/edecarlo/para-atm-collection/miscellaneous/gnats-fpgen'

trx_fname = '/iff_to_gnats_geo'
mfl_file= trx_dir+trx_fname+'_mfl.trx'
trx_file = trx_dir+trx_fname+'.trx'
results_file = results_dir+trx_fname+'.csv'

if os.path.exists(trx_file):
    os.remove(trx_file)
    print('Previously generated trx file removed...')

if os.path.exists(mfl_file):
    os.remove(mfl_file)
    print('Previously generated mfl file removed...')

In [8]:
import time
import numpy as np
from jpype import JPackage

TRACK_TIME = time.time()
clsGeometry = JPackage('com').osi.util.Geometry

for i,callsign in enumerate(cs[0:20]):
    flightInAir = check_if_flight_has_departed(iff_data,callsign,natsSim,iffBaseAirport)
    flightLandingInData = check_if_flight_landing_in_dataset(iff_data,callsign,natsSim,iffBaseAirport)
    
    arrivalAirport = ''
    departureAirport=''
    arrivalRwy = ''
    arrivalGate = ''
    departureRwy =''
    departureGate = ''
    result_generated = ''

    if flightInAir and flightLandingInData:
        #Get departure airport. If none is known in the IFF+ASDEX file (i.e., it is not the airport whose name is in the iff_fname), then set departure airport as closest airport to the first lat/lon in the dataset. In the future, would like to use IFF_USA to determine departureAirport in this case
        arrivalAirport = 'KSFO'
        departureAirport = get_departure_airport_from_iff(iff_data,callsign,natsSim,arrivalAirport=iffBaseAirport,flmap=f.flmap)
        #departureAirport = 'KEWR'
        arrivalGate,arrivalRwy= get_arrival_gate_and_rwy_from_iff(iff_data,callsign,natsSim,arrivalAirport)
        result_generated = f.generate(4, departureAirport, arrivalAirport, "", arrivalGate, "", arrivalRwy)
        

        lat = iff_data[3].loc[iff_data[3].callsign==callsign,'latitude'].iloc[0]
        lon = iff_data[3].loc[iff_data[3].callsign==callsign,'longitude'].iloc[0]
        latstr = clsGeometry.convertLatLonDeg_to_degMinSecString(str(lat))
        lonstr = clsGeometry.convertLatLonDeg_to_degMinSecString(str(lon))
        elev = np.max([iff_data[3].loc[iff_data[3].callsign==callsign,'altitude'].iloc[0]/100.,100.])
        spd = iff_data[3].loc[iff_data[3].callsign==callsign,'tas'].iloc[0]
        hdg = iff_data[3].loc[iff_data[3].callsign==callsign,'heading'].iloc[0]
        aircraftType = 'B744'
        
    if  not flightInAir and not flightLandingInData:
        departureAirport = 'KSFO'
        arrivalAirport = get_arrival_airport_from_iff(iff_data,callsign,natsSim,departureAirport,f.flmap)
        #arrivalAirport = 'KEWR'
        departureGate,departureRwy = get_departure_gate_and_rwy_from_iff(iff_data,callsign,natsSim,departureAirport) #doesn't exist
        arrivalGate,arrivalRwy = random_airport_gate_and_rwy(natsSim,arrivalAirport)
        result_generated = f.generate(1, departureAirport, arrivalAirport, departureGate, arrivalGate, departureRwy, arrivalRwy)
        
        airportInstance = natsSim.airportInterface.select_airport(departureAirport)
        elev = airportInstance.getElevation()/100.0  
        lat = iff_data[3].loc[iff_data[3].callsign==callsign,'latitude'].iloc[0]
        lon = iff_data[3].loc[iff_data[3].callsign==callsign,'longitude'].iloc[0]
        latstr = clsGeometry.convertLatLonDeg_to_degMinSecString(str(lat))
        lonstr = clsGeometry.convertLatLonDeg_to_degMinSecString(str(lon))
        aircraftType = 'B744'
        spd = 0
        hdg = 28
    
    if result_generated:
        TRACK_TIME += 10
        track_string = '%s %s %.4f %.4f %d %.2f %d %s %s'.format(callsign,aircraftType,float(latstr),float(lonstr),spd,elev,hdg,'ZOA','ZOA46')
        fp_route = result_generated[0]
        fp_validated = natsSim.aircraftInterface.validate_flight_plan_record(track_string,fp_route,300)
        if fp_validated:
            print('Validated Flight Plan:',callsign)
            print(result_generated)
            with open(trx_file,'a') as trxFile:
                tstr = '%s %s' % ('TRACK',track_string)
                trxFile.write('%s %d' % ('TRACKTIME', TRACK_TIME) + '\n')
                trxFile.write( tstr + '\n')
                trxFile.write('    FP_ROUTE ' + fp_route + '\n\n')

            with open(mfl_file,'a') as mflFile:
                mflFile.write(callsign + ' ' + '330' + '\n')

No viable origin airport found for DAL2312. Returning random from FlightPlanSelector options.
No viable destination airport found for UAL2377. Returning random from FlightPlanSelector options.
OptimalTakeoff is: I28R
Length of terminalProcedure is  4
Selected Takeoff: I28R
No viable destination airport found for UAL863. Returning random from FlightPlanSelector options.
OptimalTakeoff is: I28L
Length of terminalProcedure is  4
Selected Takeoff: I28L
No viable destination airport found for DAL806. Returning random from FlightPlanSelector options.
OptimalTakeoff is: I19L
Length of terminalProcedure is  4
Selected Takeoff: I19L
No viable origin airport found for UAL748. Returning random from FlightPlanSelector options.
No viable origin airport found for UAL210. Returning random from FlightPlanSelector options.
No viable origin airport found for JBU1715. Returning random from FlightPlanSelector options.
No viable destination airport found for UAL2732. Returning random from FlightPlanSelecto

TypeError: cannot unpack non-iterable NoneType object

In [1]:
#Filenames to write modified trx and mfl files to. Need full path because GNATS wrapper changes the directory to the GNATS directory.
import os
home_env = os.environ.get('HOME')
trx_dir = '/home/edecarlo/para-atm-collection/miscellaneous/gnats-fpgen'
results_dir = '/home/edecarlo/para-atm-collection/miscellaneous/gnats-fpgen'

trx_fname = '/iff_to_gnats_geo'
mfl_file= trx_dir+trx_fname+'_mfl.trx'
trx_file = trx_dir+trx_fname+'.trx'
results_file = results_dir+trx_fname+'.csv'

data=natsSim(trx_name=trx_file,mfl_name=mfl_file)['trajectory']
natsSim.write_output(results_file)
natsSim.cleanup()

NameError: name 'natsSim' is not defined

In [2]:
from paraatm.plotting import plot_trajectory
plot_trajectory(data, output_notebook=True, plot_width=600, plot_height=400)

NameError: name 'data' is not defined

In [11]:
data.status.unique()

array(['FLIGHT_PHASE_TAXI_DEPARTING', 'FLIGHT_PHASE_ORIGIN_GATE',
       'FLIGHT_PHASE_PUSHBACK', 'FLIGHT_PHASE_RAMP_DEPARTING',
       'FLIGHT_PHASE_RUNWAY_THRESHOLD_DEPARTING', 'FLIGHT_PHASE_TAKEOFF',
       'FLIGHT_PHASE_CLIMBOUT', 'FLIGHT_PHASE_CLIMB_TO_CRUISE_ALTITUDE',
       'FLIGHT_PHASE_CRUISE', 'FLIGHT_PHASE_INITIAL_DESCENT',
       'FLIGHT_PHASE_APPROACH', 'FLIGHT_PHASE_TOUCHDOWN',
       'FLIGHT_PHASE_OUTOFRUNWAY'], dtype=object)

In [8]:
natsSim.aircraftInterface.validate_flight_plan_record()

RuntimeError: No matching overloads found for validate_flight_plan_record in find. at native/common/jp_method.cpp:127